<a href="https://colab.research.google.com/github/rajeshreddy10/MLA04-DEEP_LEARNING/blob/main/DL_Comprehensive_Pretrained_CNN_Model_Performance_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

9. Using a Python code implementation, demonstrate the performance of pre-trained ResNet101V2 model using a chosen data of your choice. Assign the following values for the model: Use a data split of: 7:1.5:1.5 (Training:Validation:Testing); Optimizer: SGD; Pooling: avg; Epochs: 25, Initial Activation: PReLU, Final Activation: SoftMax. Draw and discuss the attained results.

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications import ResNet101V2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split


(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

# One-hot encode labels
y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

# Resize to ResNet expected size (224x224)
IMG_SHAPE = (224, 224, 3)
x_train_resized = tf.image.resize(x_train, (IMG_SHAPE[0], IMG_SHAPE[1]))
x_test_resized = tf.image.resize(x_test, (IMG_SHAPE[0], IMG_SHAPE[1]))

# Train/Val split (≈70% train, 15% val)
x_train, x_val, y_train, y_val = train_test_split(
    x_train_resized, y_train, test_size=0.176, random_state=42
)

print(f"Training samples: {len(x_train)}")
print(f"Validation samples: {len(x_val)}")
print(f"Testing samples: {len(x_test_resized)}")

from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
)
datagen.fit(x_train)

base_model = ResNet101V2(weights="imagenet", include_top=False, input_shape=IMG_SHAPE)

x = base_model.output
x = GlobalAveragePooling2D(name="avg_pooling")(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(10, activation="softmax", name="final_output")(x)

model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

optimizer = SGD(learning_rate=0.01, momentum=0.9)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Phase 1 Training (Frozen base) ---")
history1 = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=10,
    verbose=1,
)

for layer in base_model.layers[-100:]:
    layer.trainable = True

optimizer = SGD(learning_rate=0.0001, momentum=0.9)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

print("\n--- Phase 2 Training (Fine-tuning) ---")
history2 = model.fit(
    datagen.flow(x_train, y_train, batch_size=32),
    validation_data=(x_val, y_val),
    epochs=15,
    verbose=1,
)

test_loss, test_acc = model.evaluate(x_test_resized, y_test, verbose=1)
print(f"\n✅ Test Accuracy: {test_acc * 100:.2f}%")


MobileNet (8:1:1, Adam, ReLU → SoftMax)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Normalize
x_train, x_test = x_train/255.0, x_test/255.0

# Split: 8:1:1
val_split = int(0.1 * x_train.shape[0])
x_val = x_train[:val_split]; y_val = y_train[:val_split]
x_train_new = x_train[val_split:]; y_train_new = y_train[val_split:]

# Data augmentation
train_gen = ImageDataGenerator(rotation_range=20, width_shift_range=0.2,
                               height_shift_range=0.2, horizontal_flip=True)
val_gen = ImageDataGenerator()
test_gen = ImageDataGenerator()

# Load MobileNet
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(32,32,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen.flow(x_train_new, y_train_new, batch_size=32),
          validation_data=val_gen.flow(x_val, y_val),
          epochs=25)

# Evaluate
loss, acc = model.evaluate(test_gen.flow(x_test, y_test))
print(f"Test Accuracy: {acc:.4f}")


/tmp/ipython-input-3850953900.py:25: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(32,32,3))


17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1407/1407 ━━━━━━━━━━━━━━━━━━━━ 483s 331ms/step - accuracy: 0.4212 - loss: 1.8162 - val_accuracy: 0.4684 - val_loss: 1.8162
Epoch 2/25
  41/1407 ━━━━━━━━━━━━━━━━━━━━ 7:24 325ms/step - accuracy: 0.5349 - loss: 1.3275

AlexNet (7:1.5:1.5, SGD, PReLU → SoftMax)

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, PReLU, Dense
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

# CIFAR-10 dataset
(x, y), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x = x/255.0; x_test = x_test/255.0

# Split 7:1.5:1.5
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)
x_val, x_test_new, y_val, y_test_new = train_test_split(x_val, y_test, test_size=0.5, random_state=42)

# AlexNet model
inputs = tf.keras.Input(shape=(32,32,3))
x = Conv2D(96, (3,3), strides=1, padding='same')(inputs)
x = PReLU()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(256, (3,3), padding='same')(x)
x = PReLU()(x)
x = MaxPooling2D((2,2))(x)

x = Conv2D(384, (3,3), padding='same')(x)
x = PReLU()(x)
x = Conv2D(384, (3,3), padding='same')(x)
x = PReLU()(x)
x = Conv2D(256, (3,3), padding='same')(x)
x = PReLU()(x)
x = MaxPooling2D((2,2))(x)

x = Flatten()(x)
x = Dense(4096)(x); x = PReLU()(x); x = Dropout(0.5)(x)
x = Dense(4096)(x); x = PReLU()(x); x = Dropout(0.5)(x)
outputs = Dense(10, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=25, batch_size=32)

# Evaluate
loss, acc = model.evaluate(x_test_new, y_test_new)
print(f"Test Accuracy: {acc:.4f}")


EfficientNetB0 (8:1:1, Adam, ReLU → SoftMax)

In [ ]:
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model

# CIFAR-10 data
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Split 8:1:1
val_split = int(0.1*x_train.shape[0])
x_val, y_val = x_train[:val_split], y_train[:val_split]
x_train_new, y_train_new = x_train[val_split:], y_train[val_split:]

# Model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(32,32,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x_train_new, y_train_new, validation_data=(x_val, y_val), epochs=25, batch_size=32)

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


EfficientNetB2 (7:1.5:1.5, SGD, PReLU → SoftMax)

In [ ]:
from tensorflow.keras.applications import EfficientNetB2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split
import tensorflow as tf

(x, y), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x, x_test = x/255.0, x_test/255.0

# Split 7:1.5:1.5
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)
x_val, x_test_new, y_val, y_test_new = train_test_split(x_val, y_test, test_size=0.5, random_state=42)

base_model = EfficientNetB2(weights='imagenet', include_top=False, input_shape=(32,32,3))
x_layer = base_model.output
x_layer = GlobalAveragePooling2D()(x_layer)
x_layer = Dense(10)(x_layer)
x_layer = PReLU()(x_layer)
outputs = Dense(10, activation='softmax')(x_layer)

model = Model(inputs=base_model.input, outputs=outputs)
model.compile(optimizer=SGD(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=25, batch_size=32)

# Evaluate
loss, acc = model.evaluate(x_test_new, y_test_new)
print(f"Test Accuracy: {acc:.4f}")


 DenseNet121 (8:1:1, Adam, ReLU → SoftMax)

In [ ]:
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

# Split 8:1:1
val_split = int(0.1 * x_train.shape[0])
x_val, y_val = x_train[:val_split], y_train[:val_split]
x_train_new, y_train_new = x_train[val_split:], y_train[val_split:]

base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(32,32,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(x_train_new, y_train_new, validation_data=(x_val, y_val), epochs=25, batch_size=32)

# Evaluate
loss, acc = model.evaluate(x_test, y_test)
print(f"Test Accuracy: {acc:.4f}")


MobileNet on Forest-Fire Database (7:1.5:1.5, SGD, PReLU → SoftMax)

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, PReLU
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

# Load Forest-Fire dataset from GitHub
!git clone https://github.com/username/Forest-Fire-Dataset.git  # Replace with actual repo
data_dir = './Forest-Fire-Dataset/'

# Data generators with augmentation
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.3)
train_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                              class_mode='categorical', subset='training')
val_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                            class_mode='categorical', subset='validation')

# Load MobileNet base
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(train_gen.num_classes)(x)
x = PReLU()(x)
outputs = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=outputs)

# Compile
model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=25)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc:.4f}")


 DenseNet169 on Forest-Fire Database (8:1:1, Adam, ReLU → SoftMax)

In [ ]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                              class_mode='categorical', subset='training')
val_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                            class_mode='categorical', subset='validation')

# DenseNet169 model
base_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=25)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc:.4f}")


ResNet50 on Plant Leaf Disease Database (Default parameters, compute metrics)

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Load Plant Leaf Disease dataset
!git clone https://github.com/username/Plant-Leaf-Disease-Dataset.git  # Replace with actual repo
data_dir = './Plant-Leaf-Disease-Dataset/'

# Data generators
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                        class_mode='categorical', subset='training')
val_gen = datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                      class_mode='categorical', subset='validation')
test_gen = datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                       class_mode='categorical', subset='validation', shuffle=False)

# ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=10)

# Predict and compute metrics
y_pred = model.predict(test_gen)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_gen.classes

print(classification_report(y_true, y_pred_classes))
cm = confusion_matrix(y_true, y_pred_classes)
print("Confusion Matrix:\n", cm)


esNet101 on Forest-Fire Database (8:1:1, Adam, ReLU → SoftMax)

In [ ]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam

# Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                              class_mode='categorical', subset='training')
val_gen = train_datagen.flow_from_directory(data_dir, target_size=(128,128), batch_size=32,
                                            class_mode='categorical', subset='validation')

# ResNet101 model
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(128,128,3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(train_gen.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=preds)

# Compile
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Train
model.fit(train_gen, validation_data=val_gen, epochs=25)

# Evaluate
loss, acc = model.evaluate(val_gen)
print(f"Validation Accuracy: {acc:.4f}")
